# LSTM Regression Model

In [ ]:
from river_torch.regression import RollingRegressor
from river import metrics, compose, preprocessing
from river.datasets import Bikes
from torch import nn
from tqdm import tqdm

In [ ]:
class MyModule(nn.Module):

    def __init__(self, n_features, hidden_size=1):
        super().__init__()
        self.n_features=n_features
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size=n_features, hidden_size=hidden_size, num_layers=1)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, X, **kwargs):
        output, (hn, cn) = self.lstm(X)  # lstm with input, hidden, and internal state
        hn = hn.view(-1, self.hidden_size)
        return self.softmax(hn)

In [ ]:
dataset = Bikes()
metric = metrics.MAE()

model_pipeline = compose.Select('clouds', 'humidity', 'pressure', 'temperature', 'wind')
model_pipeline |= preprocessing.StandardScaler()
model_pipeline |= RollingRegressor(
    module=MyModule,
    loss_fn='mse',
    optimizer_fn='sgd',
    window_size=20,
    lr=1e-2,
    hidden_size=1,  # parameters of MyModule can be overwritten
    append_predict=True,
)
model_pipeline

In [ ]:
for x, y in tqdm(dataset.take(5000)):
    y_pred = model_pipeline.predict_one(x)
    metric = metric.update(y_true=y, y_pred=y_pred)
    model_pipeline = model_pipeline.learn_one(x=x, y=y)
print(f'MAE: {metric.get():.2f}')